In [ ]:
%pip install -e ..
%restart_python

In [ ]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent / 'src'))

In [ ]:
import pandas as pd
import yaml
from loguru import logger
from pyspark.sql import SparkSession

from honeywell.config import ProjectConfig
from honeywell.data_processor import DataProcessor

import seaborn as sns 
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()

In [ ]:

config = ProjectConfig.from_yaml(config_path="../project_config_honeywell.yml", env="dev")
logger.info("Configuration loaded:")
logger.info(yaml.dump(config, default_flow_style=False))

# COMMAND ----------
df = pd.read_csv(config.input_data_path)
logger.info(f"Data loaded from {config.input_data_path} with shape {df.shape}")

In [ ]:

# Initialize DataProcessor
data_processor = DataProcessor(df, config, spark)

# Preprocess the data
data_processor.preprocess()

# Split the data
X_train, X_test = data_processor.split_data()
logger.info("Training set shape: %s", X_train.shape)
logger.info("Test set shape: %s", X_test.shape)

# Save to catalog
logger.info("Saving data to catalog")
data_processor.save_to_catalog(X_train, X_test)
